<a href="https://colab.research.google.com/github/fabricecordelieres/Colab-Metrology_Report_Script/blob/main/Colab-Metrology_Report_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Metrology Notebook**
---
_This scripts takes as an input a Google Drive folder in which temporal metrological data from several microscopes have been stored and builds XLSX and PPTX reports from the data.
Data should have been exported using the Fiji/ImageJ MetroloJ plugin, along side the power measurements that have been properly formatted. The following section gives information about what data structure is expected._

## **Data organization**

### Metrology folder:
---
> # *__Root_metrology_Folder__*

>> <ins>*__Microscope 1 (subfolder)__*</ins>

>>> *__Power_file__* (mandatory, see below for a full description)

>>> *__YYMMDD__* (subfolder, date at which measurement was done: should match dates in the power_file)

>>>> __pdf__ (optionnal subfolder, stores the pdf extracted by the MetroloJ plugin)

>>>> __table__ (mandatory subfolder, stores tables extracted by the MetroloJ plugin)

>>> *__YYMMDD__* (subfolder, date at which measurement was done: should match dates in the power_file)

>>>> __pdf__ (optionnal subfolder, stores the pdf extracted by the MetroloJ plugin)

>>>> __table__ (mandatory subfolder, stores tables extracted by the MetroloJ plugin)

>> <ins>*__Microscope 2 (subfolder)__*</ins>

>>> *__Power_file__* (mandatory, see below for a full description)

>>> *__YYMMDD__* (subfolder, date at which measurement was done: should match dates in the power_file)

>>>> __pdf__ (optionnal subfolder, stores the pdf extracted by the MetroloJ plugin)

>>>> __table__ (mandatory subfolder, stores tables extracted by the MetroloJ plugin)

>>> *__YYMMDD__* (subfolder, date at which measurement was done: should match dates in the power_file)

>>>> __pdf__ (optionnal subfolder, stores the pdf extracted by the MetroloJ plugin)

>>>> __table__ (mandatory subfolder, stores tables extracted by the MetroloJ plugin)
---


### Powers file:

The powers file contains all measured powers as tabulation separated values.

*__The filename should be puissance.csv.__*

There is only one puissance.csv file per microscope folder, stored at its root.

The file is assumed to contain the following columns:
* *__date__*: should be formatted as YYMMDD, not forgetting the leading zeros
* *__source__*: name of the light source (LED, laser etc)
* *__obj__*: objective that was used for measurements
* *__Channel_405__*: illumination setting used for measuring the 405 channel
* *__Channel_488__*: illumination setting used for measuring the 488 channel
* *__Channel_561__*: illumination setting used for measuring the 561 channel
* *__Channel_633__*: illumination setting used for measuring the 633 channel
* *__405__*: power measured using the Channel 405 illumination setting (in mW)
* *__488__*: power measured using the Channel 488 illumination setting (in mW)
* *__561__*: power measured using the Channel 561 illumination setting (in mW)
* *__633__*: power measured using the Channel 633 illumination setting (in mW)

Data are stored, one date per line

*__ex:__*

---

date	source	obj	Channel_405	Channel_488	Channel_561	Channel_633	405	488	561	633

201201	LED	10x	0	Eye-GFP_50pct	0	0	0.000	18.800	0.000	0.000

201218	LED	10x	Eye-DAPI_50pct	Eye-GFP_50pct	Eye-TRITC_50pct	Fluo - CY5 quad quad	51.300	17.300	16.470	11.170

210127	LED	10x	Eye-DAPI_50pct	Eye-GFP_50pct	Eye-TRITC_50pct	Fluo - CY5 quad quad	43.5	15.12	14.96	9.17

---




## **How to run this script**

1.   Have all you data ready on the Google Drive
2.   Run all the steps: Run/Run all
  1. Step 1 only is interactive: it will require that acces is given by the Google Drive to the script. Simply follow the procedure explained at Step 1.1
  2. The metrology root folder should be defined: follow the procedure explained at Step 1.2
  3. All subsequent steps are run without any interaction with the user
3. At the end of the script, two files are generated in the metrology root folder:
  1. An XLSX file containing all the data pulled into a single, multitabs file
  2. A PPTX file containing graphs for all microscopes/dates/measurements

#**Step 1: Preparing the analysis, setting up some tools**

In [47]:
#@markdown ## **_Step 1.1: Mount Google Drive_**
#@markdown 1. Execute the following cell
#@markdown 2. Click on the link to generate an authorization code
#@markdown 3. Copy the code from the new window to the dedicated box

drive.mount('/content/gdrive', force_remount=True)


Mounted at /content/gdrive


In [48]:
#@markdown ## **_Step 1.2: Define the input folder_**
#@markdown 1. Go to the left side, click on the folder icon
#@markdown 2. Find the Metrology folder
#@markdown 3. Right-click on dots
#@markdown 4. Click to copy the path
#@markdown 5. Paste it to the metroRoot field

metroRoot = "/content/gdrive/MyDrive/Metrology" #@param {type:"string"}

if not metroRoot.endswith('/'):
  metroRoot+='/'

# **Step 2: Define some classes to handle the data**
_Simply execute all cells_

In [94]:
#@markdown ## **_Step 2.1: Import dependencies and define some variables_**
#@markdown _Simply execute the following cell_

!pip install xlsxwriter
!pip install python-pptx

from google.colab import drive
import os
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

from pptx import Presentation
from pptx.util import Inches
from pptx.util import Pt
from io import BytesIO

#Default variables
debug=True

#Some helpful formatting parameters
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [50]:
#@markdown ## **_Step 2.2: Define the Power_data class_**

#--------------------- HANDLE LASER POWER DATA ---------------------
class Power_data:
  """Build a Power_data object, from an input path, 
  and allows easy retrieval of specific data

  Keyword argument:
    path -- path to the Power data file"""
  
  
  ##Class attibute
  wavelengths=['405', '488', '561', '633']
    

  def __init__(self, path=''):
    self.data=pd.read_csv(path, sep='.', delimiter='\t')


  def getDates(self):
    """Returns the dates, as a list string"""
    return list(self.data['date'])


  def getPowers(self, waveName=''):
    """Returns the powers for the input waveName, as a list float or list of 0 if not found"""
    if waveName in list(self.data.columns):
      return list(map(lambda x: float(x), self.data[waveName]))
    else:
      return [0]*len(self.getDates())


  def getDataForReport(self):
    """Returns a pandas DataFrame containing the formatted powers"""
    out=pd.DataFrame({'Date': self.getDates()})

    for wave in self.wavelengths:
     out=pd.concat([out, pd.DataFrame({wave: self.getPowers(wave)})], axis=1)
     
    if 'puissance' in list(self.data.columns):
      out['488']=self.getPowers('puissance')

    return out


  def getWavelengths(self):
    """Returns the wavelengths, as a list string"""
    return self.wavelengths


In [51]:
#@markdown ## **_Step 2.3: Define the PSF_data class_**


#--------------------- HANDLE PSF DATA ---------------------
class PSF_data:
  """Build a PSF_data object, from an input path, 
  and allows easy retrieval of specific data

  Keyword argument:
    path -- path to the PSF data file"""
  
  #Class attibute
  data=pd.DataFrame()

  def __init__(self, path=''):
    self.data=pd.read_csv(path, sep='.', delimiter='\t', encoding='Latin-1')

  def getResolutionX(self):
    """Returns the measured resolution along the X axis, as a float"""
    return self.removeUnits(self.data.at[0,"FWHM"])
  
  def getResolutionY(self):
    """Returns the measured resolution along the Y axis, as a float"""
    return self.removeUnits(self.data.at[1,"FWHM"])

  def getResolutionZ(self):
    """Returns the measured resolution along the Z axis, as a float"""
    return self.removeUnits(self.data.at[2,"FWHM"])

  def getResolution(self, axis=''):
    """Returns the measured resolution along the input axis, as a float"""
    if axis=='x':
      return self.getResolutionX()
    
    if axis=='y':
      return self.getResolutionY()
    
    if axis=='z':
      return self.getResolutionZ()

    return 0.0

  def getRefResolutionX(self):
    """Returns the reference resolution along the X axis, as a float"""
    return self.removeUnits(self.data.at[0,"Theoretical resolution"])
  
  def getRefResolutionY(self):
    """Returns the reference resolution along the Y axis, as a float"""
    return self.removeUnits(self.data.at[1,"Theoretical resolution"])

  def getRefResolutionZ(self):
    """Returns the reference resolution along the Z axis, as a float"""
    return self.removeUnits(self.data.at[2,"Theoretical resolution"])

  def getRefResolution(self, axis=''):
    """Returns the reference resolution along the input axis, as a float"""
    if axis=='x':
      return self.getRefResolutionX()
    
    if axis=='y':
      return self.getRefResolutionY()
    
    if axis=='z':
      return self.getRefResolutionZ()

    return 0.0

  def getQCFitX(self):
    """Returns the fit goodness from fitting the resolution plot 
    along the X axis, as a float"""
    return float(self.data.at[0,"Fit goodness"])
  
  def getQCFitY(self):
    """Returns the fit goodness from fitting the resolution plot 
    along the Y axis, as a float"""
    return float(self.data.at[1,"Fit goodness"])

  def getQCFitZ(self):
    """Returns the fit goodness from fitting the resolution plot 
    along the Y axis, as a float"""
    return float(self.data.at[2,"Fit goodness"])
  
  def getQCFit(self, axis=''):
    """Returns the fit goodness from fitting the resolution plot 
    along the input axis, as a float"""
    if axis=='x':
      return self.getQCFitX()
    
    if axis=='y':
      return self.getQCFitY()
    
    if axis=='z':
      return self.getQCFitZ()

    return 0.0

  # Helper function
  def removeUnits(self, input_string=''):
    """Helper function: splits an input string and parse the first
    element to float"""
    return float(input_string.split()[0])

In [52]:
#@markdown ## **_Step 2.4: Define the Homogeneity_data class_**


#--------------------- HANDLE HOMOGENEITY DATA ---------------------
class Homogeneity_data:
  """Build a Homogeneity_data object, from an input path, 
  and allows easy retrieval of specific data

  Keyword argument:
    path -- path to the Homogeneity data file"""
  

  #Class attibute
  data=pd.DataFrame()


  def __init__(self, path=''):
    self.data=pd.read_csv(path, sep='.', delimiter='\t', encoding='Latin-1').iloc[0:2,1:5]
    

  def getImageCenter(self):
    """Returns the coordinates of the image's centre, as a list of float"""
    return self.stringToCoordinates(self.data.at[0,"Image centre"])
  

  def getCentreOfIntensity(self):
    """Returns the coordinates of the centre of intensity, as a list of float"""
    return self.stringToCoordinates(self.data.at[0,"Centre of intensity"])


  def getCentreOfMaxIntensity(self):
    """Returns the coordinates of the centre of max intensity, as a list of float"""
    return self.stringToCoordinates(self.data.at[0,"Centre of the max intensity"])


  def getCentreOfHundredPctZone(self):
    """Returns the coordinates of the centre of the 100% zone, as a list of float"""
    return self.stringToCoordinates(self.data.at[0,"Centre of the 100% zone"])
  

  def getDistanceToCentreOfIntensity(self):
    """Returns the distance between the image's centre and the centre of 
    intensity, as a float"""
    return self.removeUnits(self.data.at[1,"Centre of intensity"])


  def getDistanceToCentreOfMaxIntensity(self):
    """Returns the distance between the image's centre and the centre of 
    max intensity, as a float"""
    return self.removeUnits(self.data.at[1,"Centre of the max intensity"])


  def getDistanceToCentreOfHundredPctZone(self):
    """Returns the distance between the image's centre and the centre of 
    the 100% zone, as a float"""
    return self.removeUnits(self.data.at[1,"Centre of the 100% zone"])

  # Helper functions
  def removeUnits(self, input_string=''):
    """Helper function: splits an input string and parse the first
    element to float"""
    return float(input_string.replace('µm', '').replace('microns', '').replace('micron', '').replace('Â', ''))

  def stringToCoordinates(self, input_string=''):
    """Helper function: splits an input string supposed to be formatted as
    (x,y) and parse the first two elements to int"""
    
    #Remove the parenthesis, split the string then cast to float
    return list(map(lambda x: float(x), input_string.strip("()").split(", ")))

In [53]:
#@markdown ## **_Step 2.5: Define the Coalignment_data class_**


#--------------------- HANDLE CO-ALIGNEMENT DATA ---------------------
class Coalignment_data:
  """Build a Coalignment_data object, from an input path, 
  and allows easy retrieval of specific data

  Keyword argument:
    path -- path to the Coalignment data file"""
  

  #Class attibute
  data=pd.DataFrame()
  nChannels=-1


  def __init__(self, path=''):
    nLines=self.countLine(path)
    self.nChannels=int((nLines-8)/3) #8 rows for headers and spaces, 3 tables
    #self.data=pd.read_csv(path, sep='.', delimiter='\t', skiprows=nLines-(self.nChannels+2)+1)
    self.data=pd.read_csv(path, sep='.', delimiter='\t', skiprows=nLines-(self.nChannels+2)+1, encoding='Latin-1')


  def getC1_C2Dist(self):
    """Returns the distance between channels 1 & 2, as a float"""
    return self.stringToDistances(self.data.at[1,"Red"])[0]


  def getC1_C3Dist(self):
    """Returns the distance between channels 1 & 3, as a float"""
    return self.stringToDistances(self.data.at[2,"Red"])[0]


  def getC2_C3Dist(self):
    """Returns the distance between channels 1 & 3, as a float"""
    if self.nChannels<3:
      return pd.NA
    else:
      return self.stringToDistances(self.data.at[2,"Green"])[0]


  def getC1_C2RefDist(self):
    """Returns the reference distance between channels 1 & 2, as a float"""
    return self.stringToDistances(self.data.at[1,"Red"])[1]
  

  def getC1_C3RefDist(self):
    """Returns the reference distance between channels 1 & 3, as a float"""
    return self.stringToDistances(self.data.at[2,"Red"])[1]
  

  def getC2_C3RefDist(self):
    """Returns the reference distance between channels 1 & 3, as a float"""
    if self.nChannels<3:
      return pd.NA
    else:
      return self.stringToDistances(self.data.at[2,"Green"])[1]


  def getDataForReport(self, tag=''):
    """Returns a pandas DataFrame containing the formatted Co-alignement data"""
    return pd.DataFrame({tag+'_Dist C1-C2': [self.getC1_C2Dist()],
                      tag+'_Ref Dist C1-C2': [self.getC1_C2RefDist()],
                      tag+'_Dist C1-C3': [self.getC1_C3Dist()],
                      tag+'_Ref Dist C1-C3': [self.getC1_C3RefDist()],
                      tag+'_Dist C2-C3': [self.getC2_C3Dist()],
                      tag+'_Ref Dist C2-C3': [self.getC2_C3RefDist()]})
  

  def getEmptyDataForReport(self, tag=''):
    """Returns a pandas DataFrame containing the formatted Co-alignement data with NA everywhere"""
    return pd.DataFrame({tag+'_Dist C1-C2': [pd.NA],
                      tag+'_Ref Dist C1-C2': [pd.NA],
                      tag+'_Dist C1-C3': [pd.NA],
                      tag+'_Ref Dist C1-C3': [pd.NA],
                      tag+'_Dist C2-C3': [pd.NA],
                      tag+'_Ref Dist C2-C3': [pd.NA]})
  

  # Helper functions
  def stringToDistances(self, input_string=''):
    """Helper function: splits an input string supposed to be formatted as
    distance (reference_distance) and returns a list of floats"""
    
    #Remove the parenthesis, split the string then cast to float
    return list(map(lambda x: float(x), input_string.strip(")").split(" (")))


  def countLine(self, input_path=''):
    """From the input_path, counts the number of lines"""
    #with open(input_path) as f:
    with open(input_path, encoding="Latin-1") as f:
        return sum(1 for line in f)

In [54]:
#@markdown ## **_Step 2.6: Define the Metrology_Data class_**

#--------------------- GENERATE THE METROLOGY DATA ---------------------
class Metrology_Data:
  """Build a Metrology Data handler, from an input path

  Keyword argument:
    path -- path to the data folder"""
  
  #Class attibute
  metrologyRoot=''
  microscopesList=list()
  dates=list()

  coaData={}; # Dict containing coalignment data as key: values, microscope:data
  homoData={}; # Dict containing homogeneity data as key: values, microscope:data
  powerData={}; # Dict containing power data as key: values, microscope:data
  psfData={}; # Dict containing psf data as key: values, microscope:data
  
  tablesSubFolder='/table/'
  
  #Coalignement radicals
  coaRadicals=['COA-BGR', 'COA-GRIR']
  
  #Homogeneity parameter to export
  homoParam='Dist_to_100pct_zone'

  #PSF resolutions to explore
  psfRadicals=['x', 'y', 'z']
  expectedNbPSFs=3

  #Debug tag
  debug=False



  def __init__(self, path=''):
    self.metrologyRoot=path
    self.microscopesList=self.getSubFolderList(path) 
  
  def pullAll(self):
    ''' Pull all data '''
    self.pullCoAlignementData()
    self.pullHomogeneityData()
    self.pullPowerData()
    self.pullPSFData()


  def pullCoAlignementData(self):
    ''' Pull coalignement data from all the microscopes, all the date in the metrology root folder'''
    for microscope in self.microscopesList:
      #Current microscope folder
      currMicroFolder=metroRoot+microscope+'/'

      #Get dates list
      datesList=self.getSubFolderList(currMicroFolder)

      #Current microscope data
      currMicroData=pd.DataFrame()

      for currDate in datesList:
        currPathMicroDate=currMicroFolder+currDate+self.tablesSubFolder #Path microscope+date
        currCoa=pd.DataFrame() #Storage for current microscope

        currDateDataFrame=pd.DataFrame({'Date':[int(currDate)]})
        for radical in self.coaRadicals:
          if self.debug:
            print("Co-alignement pulling: "+microscope+"/"+currDate+"/"+radical)
        
          #Deal with all the coalignement file
          coaList=self.getSpecificFileList(currPathMicroDate, radical) #List of coalignement files
          if len(coaList)>0:
            coaPath=currPathMicroDate+coaList[0]
            if os.path.exists(coaPath):
              coaFile=Coalignment_data(coaPath)
              tmp=pd.DataFrame(coaFile.getDataForReport(radical)) 
            else:
              tmp=pd.DataFrame(coaFile.getEmptyDataForReport(radical)) 
          else:
              tmp=pd.DataFrame(coaFile.getEmptyDataForReport(radical))
          
          currDateDataFrame=pd.concat([currDateDataFrame, tmp], axis=1) #data from the current coalignement file
        currMicroData=pd.concat([currMicroData, currDateDataFrame], axis=0)
      self.coaData[microscope]=currMicroData


  def pullHomogeneityData(self):
    ''' Pull homogeneity data from all the microscopes, all the date in the metrology root folder'''
    for microscope in self.microscopesList:
      #Current microscope folder
      currMicroFolder=metroRoot+microscope+'/'

      #Get dates list
      datesList=self.getSubFolderList(currMicroFolder)

      #Current microscope data
      currMicroData=pd.DataFrame()

      for currDate in datesList:
        if self.debug:
          print("Homogeneity pulling: "+microscope+"/"+currDate)
        
        currPathMicroDate=currMicroFolder+currDate+self.tablesSubFolder #Path microscope+date
        
        #Deal with all the homogeneity file
        homoList=self.getSpecificFileList(currPathMicroDate, 'HOMO_') #List of homogeneity files
        if len(homoList)>0:
          homoPath=currPathMicroDate+homoList[0]
          if os.path.exists(homoPath):
            homoFile=pd.DataFrame({'Date': [int(currDate)], self.homoParam: [Homogeneity_data(homoPath).getDistanceToCentreOfHundredPctZone()]})
          else:
            homoFile=pd.DataFrame({'Date': [int(currDate)], self.homoParam: [0]})
        else:
            homoFile=pd.DataFrame({'Date': [int(currDate)], self.homoParam: [0]})
        
        currMicroData=pd.concat([currMicroData, homoFile], axis=0) #pull the the current homogeneity for current time data
      self.homoData[microscope]=currMicroData


  def pullPowerData(self):
    ''' Pull power data from all the microscopes in the metrology root folder'''
    for microscope in self.microscopesList:
      if self.debug:
        print("Powers pulling: "+microscope)
      #Current power file
      powFile=metroRoot+microscope+'/puissance.txt'
      
      if os.path.exists(powFile):
        #Read the laser power data for current microscope
        pow=Power_data(powFile)
        #Add data to dictionnary
        self.powerData[microscope]=pow.getDataForReport()


  def pullPSFData(self):
    ''' Pull homogeneity data from all the microscopes, all the date in the metrology root folder'''
    for microscope in self.microscopesList:
      #Current microscope folder
      currMicroFolder=metroRoot+microscope+'/'

      #Get dates list
      datesList=self.getSubFolderList(currMicroFolder)

      #Current microscope data
      currMicroData=pd.DataFrame()

      for currDate in datesList:
        currPathMicroDate=currMicroFolder+currDate+self.tablesSubFolder #Path microscope+date

        #Current date data
        currDateData=pd.DataFrame({'Date': [int(currDate)]})
        
        #Deal with all the psf file
        for radical in self.psfRadicals:
          if self.debug:
            print("PSF pulling: "+microscope+"/"+currDate+"/"+radical)
          currRadicalData=pd.DataFrame() #Stores data for the current radical
          psfList=self.getSpecificFileList(currPathMicroDate, 'PSF_') #List of psf files
          index=1
          
          for psf in psfList:
            psfPath=currPathMicroDate+psf
            if os.path.exists(psfPath):
              psfFile=pd.DataFrame({'Resolution_'+radical+'_PSF_'+str(index): [PSF_data(psfPath).getResolution(radical)]})
            else:
              psfFile=pd.DataFrame({'Resolution_'+radical+'_PSF_'+str(index): [pd.NA]})
            currRadicalData=pd.concat([currRadicalData, psfFile], axis=1)
            index+=1
            if index>3:
               break
            
          #Complements the list of PSFs up to the expected number of PSFs
          for i in range(index, self.expectedNbPSFs+1):
            currRadicalData=pd.concat([currRadicalData, pd.DataFrame({'Resolution_'+radical+'_PSF_'+str(i): [pd.NA]})], axis=1)
          
          #Adds the stats at the end of the current dataset
          currRadicalData=self.addMeanSD(currRadicalData, 'Resolution_'+radical+'_')
          currDateData=pd.concat([currDateData, currRadicalData], axis=1)

        currMicroData=pd.concat([currMicroData, currDateData]) #pull the the current homogeneity for current time data
        
      self.psfData[microscope]=currMicroData

  # Helper functions
  def getSubFolderList(self, path=''):
    out=os.listdir(path)
    #In case some temporary python files are there
    return sorted([f for f in out if (not f=='.ipynb_checkpoints') and os.path.isdir(path+f)])


  def getSpecificFileList(self, path='', prefix=''):
    out=os.listdir(path)
    return [f for f in out if f.startswith(prefix)]


  def addMeanSD(self, df='', suffix=''):
    dfMean=df.mean(axis=1)
    dfMean.name=suffix+'Mean'
    dfStd=df.std(axis=1)
    dfStd.name=suffix+'SD'

    df=pd.concat([df, dfMean], axis=1)
    df=pd.concat([df, dfStd], axis=1)

    return df

In [55]:
#@markdown ## **_Step 2.7: Define the Metrology_to_XLSX class_**

#--------------------- GENERATE THE METROLOGY TO XLSX ---------------------
class Metrology_to_XLSX:
  """Build a Metrology to XLSX object to generate a XLSX report from the input metrology_data

  Keyword argument:
  metroData -- a metrology_data object"""
  
  #Class attibute
  metrologyData=''

  def __init__(self, metroData=''):
    self.metrologyData=metroData


  def get_XLSX(self):
    #Generate timestamp
    timeStamp=dt.datetime.now().strftime("%y%m%d_%H:%M:%S_")


    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(self.metrologyData.metrologyRoot+timeStamp+'Metrology_report.xlsx', engine='xlsxwriter')
    workbook=writer.book
    self.addSheet(writer, workbook, 'Puissance', self.metrologyData.powerData)
    self.addSheet(writer, workbook, 'Co-alignement', self.metrologyData.coaData)
    self.addSheet(writer, workbook, 'Homogénéité', self.metrologyData.homoData)
    self.addSheet(writer, workbook, 'Résolution', self.metrologyData.psfData)

    # Close the Pandas Excel writer and output the Excel file.
    writer.save()
  

  def addSheet(self, writer='', workbook='', title='', data=''):
    worksheet=workbook.add_worksheet(title)
    writer.sheets[title]=worksheet

    # Create a format to use in the merged range.
    merge_format = workbook.add_format({
        'bold': 1,
        'border': 1,
        'bottom': 1,
        'top': 1,
        'left': 1,
        'right': 1,
        'align': 'center',
        'valign': 'vcenter',
        #'fg_color': 'yellow'
        })
    std_format = workbook.add_format({
        'bold': 0,
        'border': 1,
        'bottom': 1,
        'top': 1,
        'left': 1,
        'right': 1,
        'align': 'center',
        'valign': 'vcenter',
        #'fg_color': 'yellow'
        })

    index=0
    # Write each dataframe to a different worksheet.
    for k,v in data.items():
      nCol=len(v.columns)
      worksheet.merge_range(0, index, 0, index+nCol-1, k, merge_format)
      v.to_excel(writer, sheet_name=title, index=False, startrow=1, startcol=index)
      nRows=v.count()
      heads=v.head()

      #Adapt column width to the content (headers)
      h_index=0
      width=0;
      for head in heads:
        worksheet.set_column(h_index+index, h_index+index, len(head)+2, std_format)
        if h_index==0:
          width=len(head)+2
        h_index+=1
      
      worksheet.set_column(index, index, width, merge_format)

      index+=nCol

In [107]:
#@markdown ## **_Step 2.8: Define the Metrology_to_PPTX class_**

#--------------------- GENERATE THE METROLOGY TO PPTX ---------------------
class Metrology_to_PPTX:
  """Build a Metrology to PPTX object to generate a PPTX report from the input metrology_data

  Keyword argument:
    metroData -- a metrology_data object"""
  
  #Class attibute
  metrologyData=''
  prs=''

  def __init__(self, metroData=''):
    self.metrologyData=metroData


  def addGlobalSection(self, intermediate_slide_title, tags, graphs_titles, y_label, data):
    ''' Adds a new global section to the current PPTX report,
        presenting all microscopes on one graph
        
        Keywords arguments:
    
        intermediate_slide_title -- title of the slide that will be inserted in the PPTX file before graphs.
        tags -- a list of the parameters to be plotted, one per slide.
        graphs_titles -- A list of titles to be used for each graph
        y_label -- label to be added to the y-axis.
        data -- the dataframe from where data should be plotted
    '''
    
    slide = self.prs.slides.add_slide(self.prs.slide_layouts[0])
    slide.shapes.title.text=intermediate_slide_title

    index=0
    for tag in tags:
      for k, v in data.items():
        plt.plot([dt.datetime.strptime(str(d), '%y%m%d') for d in v['Date']], v[tag], marker='o', label=k)  
      plt.xlabel('Date')
      plt.xticks(rotation=30)
      plt.ylabel(y_label)
      plt.legend(fontsize='xx-small', loc='upper left')
      image_stream = BytesIO()
      plt.savefig(image_stream, dpi=300, format='png')
      plt.close()
      image_stream.seek(0)
      x, y, cx, cy = Inches(0.53), Inches(1.32), Inches(9), Inches(6.09)
      
      slide = self.prs.slides.add_slide(self.prs.slide_layouts[5])
      
      pic = slide.shapes.add_picture(image_stream, x, y, cx, cy)
      
      #Put image behind the title
      slide.shapes._spTree.remove(pic._element)
      slide.shapes._spTree.insert(2, pic._element)

      slide.shapes.title.text=graphs_titles[index]
      index+=1;

  def addPerMicroscopeSection(self, intermediate_slide_title, tags, graphs_titles, y_label, data):
    ''' Adds a new per microscope section to the current PPTX report,
        presenting one microscope per graph
        
        Keywords arguments:
    
        intermediate_slide_title -- title of the slide that will be inserted in the PPTX file before graphs.
        tags -- a list of the parameters to be plotted, one per slide.
        graphs_titles -- A list of titles to be used for each graph
        y_label -- label to be added to the y-axis.
        data -- the dataframe from where data should be plotted
    '''
    
    slide = self.prs.slides.add_slide(self.prs.slide_layouts[0])
    slide.shapes.title.text=intermediate_slide_title

    index=0
    for tag in tags:
      for k, v in data.items():
        plt.plot([dt.datetime.strptime(str(d), '%y%m%d') for d in v['Date']], v[tag], marker='o', label=k)  
        plt.xlabel('Date')
        plt.xticks(rotation=30)
        plt.ylabel(y_label)
        image_stream = BytesIO()
        plt.savefig(image_stream, dpi=300, format='png')
        plt.close()
        image_stream.seek(0)
        x, y, cx, cy = Inches(0.53), Inches(1.32), Inches(9), Inches(6.09)
        
        slide = self.prs.slides.add_slide(self.prs.slide_layouts[5])
        
        pic = slide.shapes.add_picture(image_stream, x, y, cx, cy)
        #Put image behind the title
        slide.shapes._spTree.remove(pic._element)
        slide.shapes._spTree.insert(2, pic._element)
        
        slide.shapes.title.text=graphs_titles[index]+'\n'+k 
      index+=1;


  def addPowerSection(self, type=0):
    intermediate_slide_title='Suivi des puissances'
    tags=['405', '488', '561', '633']
    graphs_titles=list(map(lambda x: 'Puissance à '+str(x)+' nm', tags))
    title_prefix='Puissance à '
    title_suffix=' (mW)'
    y_label='Puissance (mW)'
    data=self.metrologyData.powerData
    if type==0:
      self.addGlobalSection(intermediate_slide_title, tags, graphs_titles, y_label, data)
    else:
      self.addPerMicroscopeSection(intermediate_slide_title, tags, graphs_titles, y_label, data)


  def addHomogeneitySection(self, type=0):
    intermediate_slide_title='Suivi du centrage d\'illumination'
    tags=['Dist_to_100pct_zone']
    graphs_titles=['Distance au max. d\'intensité (µm)']
    y_label='Distance (µm)'
    data=self.metrologyData.homoData
    if type==0:
      self.addGlobalSection(intermediate_slide_title, tags, graphs_titles, y_label, data)
    else:
      self.addPerMicroscopeSection(intermediate_slide_title, tags, graphs_titles, y_label, data)

  def addPSFSection(self, type=0):
    intermediate_slide_title='Suivi des résolutions'
    tags=['Resolution_x_Mean', 'Resolution_y_Mean', 'Resolution_z_Mean']
    graphs_titles=['Résolution x (µm)', 'Résolution y (µm)', 'Résolution z (µm)']
    y_label='Résolution (µm)'
    data=self.metrologyData.psfData
    if type==0:
      self.addGlobalSection(intermediate_slide_title, tags, graphs_titles, y_label, data)
    else:
      self.addPerMicroscopeSection(intermediate_slide_title, tags, graphs_titles, y_label, data)

  def addCoAlignementSection(self, type=0):
    intermediate_slide_title='Suivi des coalignements'
    tags=['COA-BGR_Dist C1-C2', 'COA-BGR_Dist C1-C3', 'COA-BGR_Dist C2-C3', 'COA-GRIR_Dist C1-C2', 'COA-GRIR_Dist C1-C3', 'COA-GRIR_Dist C2-C3']
    graphs_titles=['Coalignement BGR C1-C2', 'Coalignement BGR C1-C3', 'Coalignement BGR C2-C3', 'Coalignement GRIR C1-C2', 'Coalignement GRIR C1-C3', 'Coalignement GRIR C2-C3']
    y_label='Distance (µm)'
    data=self.metrologyData.coaData
    if type==0:
      self.addGlobalSection(intermediate_slide_title, tags, graphs_titles, y_label, data)
    else:
      self.addPerMicroscopeSection(intermediate_slide_title, tags, graphs_titles, y_label, data)
  
  def get_PPTX(self):
    #Generate timestamp
    timeStamp=dt.datetime.now().strftime("%y%m%d_%H:%M:%S_")

    #Initialize the presentation
    self.prs = Presentation()

    #Add title slide
    slide = self.prs.slides.add_slide(self.prs.slide_layouts[0])
    slide.shapes.title.text='Metrology report\n'+dt.datetime.now().strftime("%y%m%d")

    #Add separator slide
    slide = self.prs.slides.add_slide(self.prs.slide_layouts[0])
    slide.shapes.title.text='Données globales'

    #Add the content
    self.addPowerSection()
    self.addHomogeneitySection()
    self.addPSFSection()
    self.addCoAlignementSection()

    #Add separator slide
    slide = self.prs.slides.add_slide(self.prs.slide_layouts[0])
    slide.shapes.title.text='Données par microscope'

    self.addPowerSection(1)
    self.addHomogeneitySection(1)
    self.addPSFSection(1)
    self.addCoAlignementSection(1)

    # Close the presentation and output the PPTX file.
    self.prs.save(self.metrologyData.metrologyRoot+timeStamp+'Metrology_report.pptx')

#**Step 3: Extracting the data and shaping the reports**
_Simply execute all cells_

In [61]:
#@markdown ## **_Step 3.1: Parsing the individual files, for all microscopes/dates_**

metroData=Metrology_Data(metroRoot)
#metroData.debug=True
metroData.pullAll()


In [62]:
#@markdown ## **_Step 3.2: Creating the XLSX report_**

xlsx=Metrology_to_XLSX(metroData)
xlsx.get_XLSX()


In [108]:
#@markdown ## **_Step 3.3: Creating the PPTX report_**

pptx=Metrology_to_PPTX(metroData)
pptx.get_PPTX()